#Install 
#1) Representation Function to Obtain Language Token Vectors
#2) Graphviz for visualization

In [ ]:
!pip install simpletransformers
!pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 823.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━

#Import Libraries -- torch and Submodules

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD

#Test Cuda Available and Assign Device to Perform Tensor Operations

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

#Preprocess String Function -- White Spaces and Lowercase Alphabets Only

In [ ]:
from string import ascii_lowercase as allowed_chars

def clean_sentence(sentence):
    
    lowercase = sentence.lower()
    chars = list(lowercase)
    clean_sentence = (''.join([char for char in chars if char in list(allowed_chars)+[' ']])).strip()
    return clean_sentence

#Sufficiently Descriptive Transformer Class

In [ ]:
from simpletransformers.language_representation import RepresentationModel

class SDT(object):

  #class constructor
  #initialize number of heads
  #initialize number of blocks
  #initialize order of moments
  #initialize model parameters
  def __init__(self,
               N_heads = 1,
               N_blocks = 1,
               order = 4,
               check_graph = False):
    
    self.repr_model = RepresentationModel(model_type="bert",
                                          model_name="bert-base-uncased",
                                          use_cuda=(device if device == 'cuda' else False))
    
    self.N_heads = N_heads
    self.N_blocks = N_blocks
    self.order = order
    self.check_graph = check_graph

    self.repr_size = len(self.repr_model.encode_sentences(["dummy"],combine_strategy="mean")[0])
    self.M = int(self.repr_size/self.N_heads) #head representation size

    self.Wq = [torch.rand(self.M,self.M,requires_grad=True) for i in range(self.N_heads)] #optional (self.M x m < self.M)
    self.Wk = [torch.rand(self.M,self.M,requires_grad=True) for i in range(self.N_heads)]
    self.Wv = [torch.rand(self.M,self.M,requires_grad=True) for i in range(self.N_heads)]
    
    self.W = [torch.rand(self.order,self.order,requires_grad=True) for i in range(self.N_heads)]

  #graph computation
  def compute_graph(self,
                    Input):
    
    #checking graph signal for Input
    Input_tokens = Input.split(' ')
    graph = {}
    n_tokens = len(Input_tokens)
        
    #compute function value for node-edge-node triplets
    for i in range(n_tokens):
      for j in range(n_tokens):

        token_i = Input_tokens[i]
        token_j = Input_tokens[j]
          
        token_i_id = token_i+str(i)
        token_j_id = token_j+str(j) 

        token_i_v = round(float(obj.forward(token_i)),2)
        token_j_v = round(float(obj.forward(token_j)),2)

        edge_weight = 1.0 - round(float(token_i_v - token_j_v),2)
          
        if ((token_j_id,token_i_id) in graph) or token_i_id == token_j_id:
          continue

        graph[(token_i_id,token_j_id)] = (str(token_i_v),str(edge_weight),str(token_j_v))

    self.graph = graph

  #parameter update equations
  def update_params(self,
                    learning_rate):
    
    for i in range(self.N_heads):
      
      self.Wq[i] += learning_rate * self.Wq[i].grad
      self.Wk[i] += learning_rate * self.Wk[i].grad
      self.Wv[i] += learning_rate * self.Wv[i].grad
      self.W[i] += learning_rate * self.W[i].grad

  #zero gradients after each gradient update
  def zero_grad(self):
      
    for i in range(self.N_heads):

      self.Wq[i].grad.zero_()
      self.Wk[i].grad.zero_()
      self.Wv[i].grad.zero_()
      self.W[i].grad.zero_()

  #Batch Normalization
  def normalize(self,X):
      
    mu = X.mean(dim=0,keepdim=True)
    sigma = X.std(dim=0,keepdim=True)
    Z = torch.nan_to_num((X-mu)/sigma)
    return Z

  #Compute k-th order moments
  def compute_moments(self,
                      embeddings):
      
    #Center the Dataset first
    X = embeddings
    Z = self.normalize(X)

    #compute upto kth order moments, count , mean , std-dev, skewness, kurtoisis, etc. ..
    Z.to(device)
    moments = []
    for k in range(self.order):
      moment_k = torch.pow(Z,k).mean(dim=0,keepdim=True)
      moments.append(moment_k[0])
    moments = torch.column_stack(moments)

    #re-center
    moments = self.normalize(moments)

    #return normalized decentralized moments
    return moments

  #Obtain Input Embeddings
  def embed_input(self,
                  Input):

    clean_Input = clean_sentence(Input)
    head_embeddings,tokens = [],clean_Input.split(' ')

    for i in range(self.N_heads):

      embeddings = []
      for token in tokens:
        embeddings.append(self.repr_model.encode_sentences([token],combine_strategy="mean")[0])      
        
      head_embeddings.append(torch.stack([torch.tensor(embedding[(i*self.M):((i+1)*(self.M))]) for embedding in embeddings]))

    head_embeddings = torch.stack(head_embeddings)
    return head_embeddings

  #Aggregate Forward Pass
  def aggregate(self,
                Input):

    head_embeddings = self.embed_input(Input)
      
    for i in range(self.N_blocks):

      #compute moments
      head_embeddings = torch.stack([self.compute_moments(head_embeddings[i]).t() for i in range(self.N_heads)])
        
      #compute Q,K,V for all attention heads
      head_Qs = torch.stack([head_embeddings[i] @ self.Wq[i] for i in range(self.N_heads)])
      head_Ks = torch.stack([head_embeddings[i] @ self.Wk[i] for i in range(self.N_heads)])
      head_Vs = torch.stack([head_embeddings[i] @ self.Wv[i] for i in range(self.N_heads)])

      #compute softmax(QK^T/sqrt_d)
      sqrt_d = torch.sqrt(torch.tensor(self.M))
      softmax = nn.Softmax(dim=1)
      head_QKs = torch.stack([softmax(torch.div(head_Qs[i] @ head_Ks[i].t(),sqrt_d)) for i in range(self.N_heads)])

      #compute weighted residual and Vs
      residuals = torch.stack([head_QKs[i] @ head_Vs[i] for i in range(self.N_heads)])
      head_Vs = torch.stack([residuals[i] + head_embeddings[i] for i in range(self.N_heads)])
    
      #batch normalize
      head_Vs = torch.stack([self.normalize(head_Vs[i]) for i in range(self.N_heads)])
      head_embeddings = head_Vs

    assoc_matrix = torch.stack([self.W[i] @ (head_embeddings[i] @ head_embeddings[i].t()) for i in range(self.N_heads)])
    mean_assoc_matrix = torch.mean(assoc_matrix,dim=0)
      
    return torch.sum(mean_assoc_matrix)

  #Forward Pass
  def forward(self,
              Input):

    output = self.aggregate(Input)
    #outputs = torch.stack([self.aggregate(token) for token in Input.split(' ')])
    #output += torch.sum(outputs @ outputs.t())
    return torch.sigmoid(output)

#Unit Tests

In [ ]:
from random import sample
from tqdm import tqdm
import pickle
#generating a pickle file
pickle_object=open('Novel_Method.pkl','ab')
#data and counts
data = [["a golden retriever is a dog",1],
        ["a cat is a dog",0]]*2

n_data = len(data)

#seed random number generator
torch.manual_seed(0)

#SDTconfig
#n_heads = 1 #-->1-12
#n_blocks = 1 #-->1-12
#order = 4 #-->0-50
check_graph = False
for n_heads_i in range(1,13):
  for n_blocks_i in range(1,13):
    for order_i in range(1,51):
      #obtain SDT object
      obj = SDT(N_heads = n_heads_i,
                N_blocks = n_blocks_i,
                order = order_i,
                check_graph = check_graph)

      #training config

      learning_rate =[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,1] #-->0.001,0.002,...0.009, 0.01,0.02,...0.09,0.1,...0.9,1.0
      n_epochs = [10,50,100,200,500] #-->10,50,100,200,500
      batch_size = [0.1,0.2,0.5,0.8,1.0] #-->0.1,0.2,0.5,0.8,1.0
      batch_size = [1.0]

      #training loop
      for learning_rate_i in learning_rate:
        for batch_size_i in batch_size:
          for n_epochs_i in n_epochs:
            for i in tqdm(range(n_epochs_i)):

              #initialize loss
              loss = torch.tensor(0.0)

              #sample batch
              batch = sample(data,int(batch_size_i*n_data))
              n_batch = len(batch)

              #calculate total loss for batch
              for i in range(n_batch):

                Input = batch[i][0]
                label = batch[i][1]

                g_truth = torch.tensor(([1.0, 0.0] if label == 1 else [0.0, 1.0]))
                output = obj.forward(Input) 
                output_probs = torch.stack([output, 
                                            1-output])
                    
                output_loss = -1 * (g_truth @ torch.log(output_probs))
                loss += output_loss

              #average loss for the batch
              loss /= n_batch

              #print loss
              #print (loss)

              #compute gradients
              loss.backward()

              #update parameters
              with torch.no_grad():
                obj.update_params(learning_rate_i)

              #zero grads out
              obj.zero_grad()

              #pickle the obj file after each epoch and write to disk
              pickle.dumps(obj,protocol=None, fix_imports=True, buffer_callback=pickle_object.write)

if obj.check_graph:

  for i in range(n_data):

    Input = data[i][0]
    obj.compute_graph(Input)
    Input_graph = obj.graph

#Graph Visualization

In [ ]:
from graphviz import Graph,Source

def visualize(Input_graph):

  g = Graph('G', filename='process.gv', engine='sfdp')

  for item in Input_graph:

    g.edge(item[0],item[1],label=Input_graph[item][1])

  return g

visualize(Input_graph)

In [ ]:
x = torch.tensor(obj.repr_model.encode_sentences(["I am a dog"],combine_strategy="mean")[0])
y = torch.tensor(obj.repr_model.encode_sentences(["I am a cat"],combine_strategy="mean")[0])
x @ y